In [2]:
import random
import os
import pandas as pd
import numpy as np
import re
import time
import requests

from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

In [3]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless=new')
chrome_options.add_argument('--no-sandbox')

driver = webdriver.Chrome(
    options=chrome_options
)


In [4]:
root_dir = './vn_news'
if not os.path.exists(root_dir):
    os.makedirs(root_dir)
n_pages = 5
article = 0

# 3. Get list of articles

In [5]:
news_page_urls = []
article_id = 0

In [14]:
for page_idx in range(n_pages):
    main_url = f'https://vietnamnet.vn/thoi-su-page{page_idx}/'
    driver.get(main_url)

    news_lst_xpath = '//div[@class="topStory-15nd"]/div/div[1]/a'
    news_tags = driver.find_elements(By.XPATH, news_lst_xpath)

    news_page_urls = [
        news_tag.get_attribute('href') for news_tag in news_tags
    ]
    for news_page_url in news_page_urls:
        driver.get(news_page_url)
        print(news_page_url)
        time.sleep(2)

        main_content_xpath = '//div[@class="content-detail content-mobile-change"]'
        print(main_content_xpath)
        try:
            main_content_tag = driver.find_element(By.XPATH, main_content_xpath)
        except:
            continue

        # Ignore video
        video_content_xpath = '//div[@class="video-detail"]'
        try:
            video_content_tag = driver.find_element(By.XPATH, video_content_xpath)
            continue
        except:
            pass
        # Get title
        title = main_content_tag.find_element(By.TAG_NAME, 'h1').text.strip()
        # Get abstract
        abstract = main_content_tag.find_element(By.TAG_NAME, 'h2').text.strip()
        # Get author
        try:
            author_xpath = '//span[@class="name"]'
            author = main_content_tag.find_element(By.XPATH, author_xpath).text.strip()
        except:
            author = ''
        # Get paragraphs
        paragraph_xpath = '//div[@class="maincontent main-content"]/p'
        paragraph_tags = main_content_tag.find_elements(By.XPATH, paragraph_xpath)

        paragraphs_lst = [
            paragraph_tag.text.strip() for paragraph_tag in paragraph_tags
        ]
        paragraphs = ' '.join(paragraphs_lst)
        final_content_lst = [title, abstract, author, paragraphs]
        final_content = '\n\n'.join(final_content_lst)
        # Save to file
        article_filename = f'article_{article_id:05d}.txt'
        article_path = os.path.join(root_dir, article_filename)

        article_id += 1
        with open(article_path, 'w', encoding='utf-8') as f:
            f.write(final_content)
        
        driver.back()


https://vietnamnet.vn/tram-nguoi-o-yen-bai-tim-kiem-be-trai-6-tuoi-di-lac-trong-rung-2-ngay-2313453.html
//div[@class="content-detail content-mobile-change"]
https://vietnamnet.vn/de-xuat-cho-phep-lao-dong-dong-tien-no-bao-hiem-that-nghiep-de-huong-che-do-2313565.html
//div[@class="content-detail content-mobile-change"]
https://vietnamnet.vn/ha-noi-tap-trung-nguon-luc-de-khoi-cong-2-tuyen-metro-hon-76-000-ty-dong-2313386.html
//div[@class="content-detail content-mobile-change"]
https://vietnamnet.vn/xu-phat-2-don-vi-lien-quan-du-an-ho-chua-nuoc-dong-thanh-sat-lo-2313500.html
//div[@class="content-detail content-mobile-change"]
https://vietnamnet.vn/da-khoi-to-1-038-bi-can-trong-vu-4-tiep-vien-hang-khong-xach-ma-tuy-ve-viet-nam-2313493.html
//div[@class="content-detail content-mobile-change"]
https://vietnamnet.vn/tron-truy-na-sang-nhat-2-nam-nguyen-thanh-dat-bi-bat-tai-san-bay-noi-bai-2313571.html
//div[@class="content-detail content-mobile-change"]
https://vietnamnet.vn/dong-nai-trien